In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import torch
import pandas as pd
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import protector as protect
from utils.cli_utils import softmax_ent

from tent import Tent, configure_model, collect_params
from typing import Sequence, Tuple, Dict, Optional
import argparse

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

from utilities import *  ## created by me
from plotting import *  ## created by me
from LabelShiftExperiments import *  ## created by me
from protector import *  ## created by me

In [3]:
CORRUPTIONS = (
    "shot_noise",
    "motion_blur",
    "snow",
    "pixelate",
    "gaussian_noise",
    "defocus_blur",
    "brightness",
    "fog",
    "zoom_blur",
    "frost",
    "glass_blur",
    "impulse_noise",
    "contrast",
    "jpeg_compression",
    "elastic_transform",
)

In [4]:
## ENTER PARAMETERS ##

# Manual settings for arguments
args = type("Args", (), {})()  # Create a simple namespace object
args.device = "cpu"  # Change this manually as needed
args.method = "none"  # Options: 'none' or 'tent'
args.corruption = "gaussian_noise"  # Choose from CORRUPTIONS
args.all_corruptions = False  # Set to True to test all corruptions
args.n_examples = 1000
args.batch_size = 64

In [5]:
# Dynamically set device to best available option
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

# Define normalization transform using CIFAR-10 mean and std values
transform = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616))

# Load pre-trained model and move to appropriate device
print("🚀 Loading model...")
model = get_model(args.method, device)

# Load clean CIFAR-10 test data to compute source entropies
print("📦 Loading clean CIFAR-10 as source entropy")
clean_ds = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transforms.Compose([transforms.ToTensor(), transform])
)
clean_loader = DataLoader(clean_ds, batch_size=args.batch_size, shuffle=False)
source_ents, accuracy, logits_list, labels_list = evaluate(model, clean_loader, device)

# Initialize protector with source entropies for shift detection
protector = protect.get_protector_from_ents(
    source_ents, argparse.Namespace(gamma=1 / (8 * np.sqrt(3)), eps_clip=1.8, device=device))

🚀 Loading model...


Using cache found in /Users/louis/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


📦 Loading clean CIFAR-10 as source entropy
Files already downloaded and verified


2025-05-10 19:35:16.659 | INFO     | protector:get_protector_from_ents:153 - creating protector from ents
2025-05-10 19:35:16.661 | INFO     | protector:set_gamma:49 - setting gamma val to 0.07216878364870323
2025-05-10 19:35:16.661 | INFO     | protector:set_eps_clip_val:45 - setting epsilon clip val to 1.8


In [19]:
# === FALSE POSITIVE RATE EVALUATION (Label Shift) ===
compare_fpr_across_seeds(
    model=model,
    load_cifar10_label_shift_balanced=load_cifar10_label_shift,
    BasicDataset=BasicDataset,
    run_martingale=run_martingale,
    protector=protector,  # Make sure this is pre-initialized with source entropies
    transform=transform,
    args=args,
    device=device,
    seeds=list(range(3)),
    num_classes_list=[1,2,3,4,5,6,7,8,9,10],
    use_pbrs=False,
    log_path='fpr_results.csv'
)


🔁 Running with seed: 0
Evaluating label shift (1-class): (2,)
Evaluating label shift (1-class): (8,)
Evaluating label shift (1-class): (4,)
Evaluating label shift (1-class): (9,)
Evaluating label shift (1-class): (1,)
Evaluating label shift (1-class): (6,)
Evaluating label shift (1-class): (7,)
Evaluating label shift (1-class): (3,)
Evaluating label shift (1-class): (0,)
Evaluating label shift (1-class): (5,)
Maximum log S_j value: 0.3115027282886838
Evaluating label shift (2-class): (4, 7)
Evaluating label shift (2-class): (5, 8)
Evaluating label shift (2-class): (6, 9)
Evaluating label shift (2-class): (2, 8)
Evaluating label shift (2-class): (8, 9)
Evaluating label shift (2-class): (4, 8)
Evaluating label shift (2-class): (0, 6)
Evaluating label shift (2-class): (0, 1)
Evaluating label shift (2-class): (2, 3)
Evaluating label shift (2-class): (3, 6)
Evaluating label shift (2-class): (0, 5)
Evaluating label shift (2-class): (1, 5)
Evaluating label shift (2-class): (3, 8)
Evaluating 

In [8]:
np.log(100)

4.605170185988092

In [12]:
compare_fpr_across_seeds(
    model=model,
    load_cifar10_label_shift_balanced=load_cifar10_label_shift,
    BasicDataset=BasicDataset,
    run_martingale=run_martingale,
    protector=protector,
    transform=transform,
    args=args,
    device=device,
    seeds=list(range(3)),
    num_classes_list=[1,2,3,4,5,6,7,8,9,10],
    use_pbrs=True,
    log_path='fpr_results_PBRS.csv'
)


🔁 Running with seed: 0
[DEBUG] Max log_sj = 4.96 for subset size 1 with classes (5,)
[DEBUG] Max log_sj = 26.12 for subset size 2 with classes (0, 8)
[DEBUG] Max log_sj = 1.61 for subset size 3 with classes (5, 6, 9)
[DEBUG] Max log_sj = 3.55 for subset size 4 with classes (0, 7, 8, 9)
[DEBUG] Max log_sj = 0.70 for subset size 5 with classes (2, 5, 6, 7, 9)
[DEBUG] Max log_sj = 1.15 for subset size 6 with classes (1, 2, 5, 6, 7, 9)
[DEBUG] Max log_sj = 1.81 for subset size 7 with classes (0, 1, 2, 3, 5, 7, 8)
[DEBUG] Max log_sj = 1.61 for subset size 8 with classes (0, 1, 2, 4, 5, 6, 7, 9)
[DEBUG] Max log_sj = 7.34 for subset size 9 with classes (0, 1, 2, 3, 5, 6, 7, 8, 9)
[DEBUG] Max log_sj = 5.81 for subset size 10 with classes (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

🔁 Running with seed: 1
[DEBUG] Max log_sj = 9.22 for subset size 1 with classes (5,)
[DEBUG] Max log_sj = 16.57 for subset size 2 with classes (4, 9)
[DEBUG] Max log_sj = 1.41 for subset size 3 with classes (0, 7, 9)
[DEBUG] Ma

In [9]:
compare_fpr_across_seeds(
    model=model,
    load_cifar10_label_shift_balanced=load_cifar10_label_shift,
    BasicDataset=BasicDataset,
    run_martingale=run_martingale,
    protector=protector,
    transform=transform,
    args=args,
    device=device,
    seeds=list(range(3)),
    num_classes_list=[1,2,3,4,5,6,7,8,9,10],
    use_pbrs=True,
    log_path='fpr_results_PBRS.csv'
)


🔁 Running with seed: 0
[DEBUG] Max log_sj = 9.75
[DEBUG] Max log_sj = 34.56
[DEBUG] Max log_sj = 1.28
[DEBUG] Max log_sj = 20.70
[DEBUG] Max log_sj = 14.42
[DEBUG] Max log_sj = 7.57
[DEBUG] Max log_sj = 0.02
[DEBUG] Max log_sj = 2.81
[DEBUG] Max log_sj = 14.70
[DEBUG] Max log_sj = 4.96
[DEBUG] Max log_sj = 0.07
[DEBUG] Max log_sj = 10.09
[DEBUG] Max log_sj = 10.87
[DEBUG] Max log_sj = 15.14
[DEBUG] Max log_sj = 21.87
[DEBUG] Max log_sj = 16.18
[DEBUG] Max log_sj = 4.87
[DEBUG] Max log_sj = 3.51
[DEBUG] Max log_sj = 1.51
[DEBUG] Max log_sj = 0.19
[DEBUG] Max log_sj = 8.49
[DEBUG] Max log_sj = 4.37
[DEBUG] Max log_sj = 4.98
[DEBUG] Max log_sj = 4.54


KeyboardInterrupt: 

In [23]:
compare_fpr_across_seeds(
    model=model,
    load_cifar10_label_shift_balanced=load_cifar10_label_shift,
    BasicDataset=BasicDataset,
    run_martingale=run_martingale,
    protector=protector,
    transform=transform,
    args=args,
    device=device,
    seeds=list(range(3)),
    num_classes_list=[1,2,3,4,5,6,7,8,9,10],
    use_pbrs=True,
    log_path='fpr_results_PBRS.csv'
)


🔁 Running with seed: 0
Evaluating label shift (1-class): (2,)
Evaluating label shift (1-class): (8,)
Evaluating label shift (1-class): (4,)
Evaluating label shift (1-class): (9,)
Evaluating label shift (1-class): (1,)
Evaluating label shift (1-class): (6,)
Evaluating label shift (1-class): (7,)
Evaluating label shift (1-class): (3,)
Evaluating label shift (1-class): (0,)
Evaluating label shift (1-class): (5,)
Maximum log S_j value: 4.96245734644752
Evaluating label shift (2-class): (4, 7)
Evaluating label shift (2-class): (5, 8)
Evaluating label shift (2-class): (6, 9)
Evaluating label shift (2-class): (2, 8)
Evaluating label shift (2-class): (8, 9)
Evaluating label shift (2-class): (4, 8)
Evaluating label shift (2-class): (0, 6)
Evaluating label shift (2-class): (0, 1)
Evaluating label shift (2-class): (2, 3)
Evaluating label shift (2-class): (3, 6)
Evaluating label shift (2-class): (0, 5)
Evaluating label shift (2-class): (1, 5)
Evaluating label shift (2-class): (3, 8)
Evaluating la

In [ ]:
# === TRUE POSITIVE RATE EVALUATION (Covariate Shift) ===
evaluate_covariate_shift_detection(
    model=model,
    load_cifar10_corruption=load_cifar10c,
    BasicDataset=BasicDataset,
    run_martingale=run_martingale,
    protector=protector,
    transform=transform,
    args=args,
    device=device,
    corruption_types=['gaussian_noise', 'fog', 'brightness', 'contrast'],  # Customize this if needed
    severities=[1, 2, 3, 4, 5],
    seeds=range(3),
    buffer_capacity=256,
    confidence_threshold=0.8,
    num_classes=10,
    use_pbrs=False,
    log_path='tpr_results.csv'
)

In [21]:
evaluate_covariate_shift_detection(
    model=model,
    load_cifar10_corruption=load_cifar10c,
    BasicDataset=BasicDataset,
    run_martingale=run_martingale,
    protector=protector,
    transform=transform,
    args=args,
    device=device,
    corruption_types=CORRUPTIONS,
    severities=[1, 2, 3, 4, 5],
    seeds=range(3),
    buffer_capacity=256,
    confidence_threshold=0.8,
    num_classes=10,
    use_pbrs=True,
    log_path='tpr_results_pbrs.csv'
)

Corruption: gaussian_noise, Severity: 1, Seed: 0
Files already downloaded and verified
Corruption: gaussian_noise, Severity: 1, Seed: 1
Files already downloaded and verified
Corruption: gaussian_noise, Severity: 1, Seed: 2
Files already downloaded and verified
✅ Detected in 0 / 3 runs (rate = 0.00)  |  Avg delay: None
Corruption: gaussian_noise, Severity: 2, Seed: 0
Files already downloaded and verified
Corruption: gaussian_noise, Severity: 2, Seed: 1
Files already downloaded and verified
Corruption: gaussian_noise, Severity: 2, Seed: 2
Files already downloaded and verified
✅ Detected in 0 / 3 runs (rate = 0.00)  |  Avg delay: None
Corruption: gaussian_noise, Severity: 3, Seed: 0
Files already downloaded and verified
Corruption: gaussian_noise, Severity: 3, Seed: 1
Files already downloaded and verified
Corruption: gaussian_noise, Severity: 3, Seed: 2
Files already downloaded and verified
✅ Detected in 0 / 3 runs (rate = 0.00)  |  Avg delay: None
Corruption: gaussian_noise, Severity: 4,

defaultdict(dict,
            {('gaussian_noise', 1): {'detection_rate': 0.0,
              'avg_delay': None,
              'raw_detections': [0, 0, 0],
              'raw_delays': [None, None, None]},
             ('gaussian_noise', 2): {'detection_rate': 0.0,
              'avg_delay': None,
              'raw_detections': [0, 0, 0],
              'raw_delays': [None, None, None]},
             ('gaussian_noise', 3): {'detection_rate': 0.0,
              'avg_delay': None,
              'raw_detections': [0, 0, 0],
              'raw_delays': [None, None, None]},
             ('gaussian_noise', 4): {'detection_rate': 0.0,
              'avg_delay': None,
              'raw_detections': [0, 0, 0],
              'raw_delays': [None, None, None]},
             ('gaussian_noise', 5): {'detection_rate': 0.0,
              'avg_delay': None,
              'raw_detections': [0, 0, 0],
              'raw_delays': [None, None, None]},
             ('fog', 1): {'detection_rate': 0.0,
        

In [24]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_detection_delay_comparison_from_csv(file_no_pbrs, file_pbrs):
    """
    Plot detection delay vs severity for each corruption type, comparing No PBRS and PBRS.
    
    Parameters:
    - file_no_pbrs: str, path to CSV file containing results without PBRS
    - file_pbrs: str, path to CSV file containing results with PBRS
    """

    # Load CSV files
    df_no = pd.read_csv(file_no_pbrs)
    df_pb = pd.read_csv(file_pbrs)

    # Construct lookup dictionaries: {(corruption, severity): avg_delay}
    def build_delay_dict(df):
        delay_dict = {}
        for _, row in df.iterrows():
            key = (row['Corruption'], int(row['Severity']))
            delay_dict[key] = row['Average Delay']
        return delay_dict

    delay_no = build_delay_dict(df_no)
    delay_pb = build_delay_dict(df_pb)

    # Determine unique corruption types
    corruptions = sorted(set([row['Corruption'] for _, row in df_no.iterrows()]))
    severities = [1, 2, 3, 4, 5]

    # Create plots
    fig, axs = plt.subplots(1, len(corruptions), figsize=(5 * len(corruptions), 4), sharey=True)
    if len(corruptions) == 1:
        axs = [axs]

    for ax, corruption in zip(axs, corruptions):
        delays_no = [delay_no.get((corruption, s), None) for s in severities]
        delays_pb = [delay_pb.get((corruption, s), None) for s in severities]

        ax.plot(severities, delays_no, marker='o', label='No PBRS')
        ax.plot(severities, delays_pb, marker='s', label='PBRS')

        ax.set_title(corruption.replace('_', ' ').title())
        ax.set_xlabel('Severity')
        ax.set_xticks(severities)
        ax.grid(True)

    axs[0].set_ylabel('Detection Delay')
    axs[0].legend()
    plt.tight_layout()
    plt.show()

In [ ]:
plot_detection_delay_comparison_from_csv("tpr_results.csv", "tpr_results_pbrs.csv")

Hyperparameter Tuning

In [22]:
buffer_sizes=[64, 128, 256, 512]
confidence_thresholds=[0.6, 0.7, 0.8]
gamma_values=[1 / (16 * 3 ** 0.5), 1 / (8 * 3 ** 0.5), 1 / (4 * 3 ** 0.5)]
eps_clip_values=[1.8, 2.0, 2.5, 3.0]

print(f"Total combinations: {len(list(itertools.product(buffer_sizes, confidence_thresholds, gamma_values, eps_clip_values)))}")

Total combinations: 144


In [24]:
(144 * 12) / 60

28.8

In [ ]:
import itertools
import pandas as pd
from datetime import datetime

def protector_factory(gamma, eps_clip, device):
    args = argparse.Namespace(gamma=gamma, eps_clip=eps_clip, device=device)
    return get_protector_from_ents(source_ents, args)  # or use dummy `source_ents` for instantiation


def combined_grid_search_pbrs(model,
                               compare_fpr_across_seeds,
                               evaluate_covariate_shift_detection,
                               load_cifar10_corruption,
                               load_cifar10_label_shift_balanced,
                               BasicDataset,
                               run_martingale,
                               protector_factory,
                               transform,
                               args,
                               device,
                               corruption_types,
                               severities,
                               seeds_tpr=range(3),
                               seeds_fpr=range(3),
                               num_classes_list=[1, 2, 3, 4, 5, 6, 7, 8, 9],
                               buffer_sizes=[64, 128, 256, 512],
                               confidence_thresholds=[0.6, 0.7, 0.8],
                               gamma_values=[1 / (16 * 3 ** 0.5), 1 / (8 * 3 ** 0.5), 1 / (4 * 3 ** 0.5)],
                               eps_clip_values=[1.8, 2.0, 2.5, 3.0],
                               log_path='combined_grid_search_results.csv'):

    results = []
    run_time = datetime.now().strftime("%Y%m%d_%H%M%S")

    for buffer_size, threshold, gamma, eps_clip in itertools.product(
        buffer_sizes, confidence_thresholds, gamma_values, eps_clip_values
    ):
        print(f"\\n🔧 Running buffer={buffer_size}, threshold={threshold}, gamma={gamma:.4f}, eps_clip={eps_clip:.2f}")

        run_id = f"{run_time}_B{buffer_size}_T{threshold}_G{gamma:.3f}_E{eps_clip:.2f}"

        # Create fresh protector instance
        protector_tpr = protector_factory(gamma=gamma, eps_clip=eps_clip, device=device)

        # --- TPR Evaluation ---
        tpr_result = evaluate_covariate_shift_detection(
            model=model,
            load_cifar10_corruption=load_cifar10_corruption,
            BasicDataset=BasicDataset,
            run_martingale=run_martingale,
            protector=protector_tpr,
            transform=transform,
            args=args,
            device=device,
            corruption_types=corruption_types,
            severities=severities,
            seeds=seeds_tpr,
            buffer_capacity=buffer_size,
            confidence_threshold=threshold,
            num_classes=10,
            use_pbrs=True,
            log_path=f'tpr_results_{run_id}.csv'
        )

        tpr_scores = [v['detection_rate'] for v in tpr_result.values()]
        delays = [v['avg_delay'] for v in tpr_result.values() if v['avg_delay'] is not None]
        mean_tpr = float(pd.Series(tpr_scores).mean())
        std_tpr = float(pd.Series(tpr_scores).std())
        mean_delay = float(pd.Series(delays).mean()) if delays else None
        std_delay = float(pd.Series(delays).std()) if delays else None


        # --- FPR Evaluation ---
        protector_fpr = protector_factory(gamma=gamma, eps_clip=eps_clip, device=device)
        
        fpr_result = compare_fpr_across_seeds(
            model=model,
            load_cifar10_label_shift_balanced=load_cifar10_label_shift_balanced,
            BasicDataset=BasicDataset,
            run_martingale=run_martingale,
            protector=protector_fpr,
            transform=transform,
            args=args,
            device=device,
            seeds=seeds_fpr,
            num_classes_list=num_classes_list,
            use_pbrs=True,
            log_path=f'fpr_results_{run_id}.csv'
        )

        fpr_scores = list(fpr_result.values())
        mean_fpr = float(pd.Series(fpr_scores).mean())
        std_fpr = float(pd.Series(fpr_scores).std())

        # Scalar tradeoff score (weights can be tuned)
        score = (0.8 * (1 - mean_fpr)) + (0.9 * mean_tpr) - (0.3 * (mean_delay / 4000)) if mean_delay is not None else None

        results.append({
            'run_id': run_id,
            'buffer_size': buffer_size,
            'confidence_threshold': threshold,
            'gamma': gamma,
            'eps_clip': eps_clip,
            'mean_fpr': mean_fpr,
            'std_fpr': std_fpr,
            'mean_tpr': mean_tpr,
            'std_tpr': std_tpr,
            'avg_delay': mean_delay,
            'std_delay': std_delay,
            'score': score
        })

        print(f"\\n🔍 Results for run_id {run_id}:")
        print(results)

    df = pd.DataFrame(results)
    df.to_csv(log_path, index=False)
    print(f"\\n✅ Combined grid search results saved to: {log_path}")


In [26]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [32]:
import optuna
import pandas as pd
from datetime import datetime

def run_optuna_pbrs_optimization(model,
                                  compare_fpr_across_seeds,
                                  evaluate_covariate_shift_detection,
                                  load_cifar10_corruption,
                                  load_cifar10_label_shift_balanced,
                                  BasicDataset,
                                  run_martingale,
                                  protector_factory,
                                  transform,
                                  args,
                                  device,
                                  corruption_types,
                                  severities,
                                  num_classes_list=[1, 2, 3, 4, 5, 6, 7, 8, 9],
                                  seeds_tpr=range(3),
                                  seeds_fpr=range(3),
                                  n_trials=50,
                                  log_path='optuna_pbrs_results.csv'):
    
    run_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    results = []

    def objective(trial):
        buffer_size = trial.suggest_categorical("buffer_size", [32, 64, 128, 256, 512, 1024])
        confidence_threshold = trial.suggest_float("confidence_threshold", 0.5, 0.95, step=0.05)
        gamma = trial.suggest_categorical("gamma", [1 / (16 * 3 ** 0.5), 1 / (8 * 3 ** 0.5), 1 / (4 * 3 ** 0.5)])
        eps_clip = trial.suggest_categorical("eps_clip", [1.5, 1.8, 2.0, 2.5, 3.0])

        run_id = f"{run_time}_B{buffer_size}_T{confidence_threshold:.2f}_G{gamma:.3f}_E{eps_clip:.2f}"
        print(f"\n🔧 Trial {trial.number}: {run_id}")

        # Protector for both TPR and FPR
        protector_tpr = protector_factory(gamma, eps_clip, device)
        protector_fpr = protector_factory(gamma, eps_clip, device)

        # --- TPR Evaluation ---
        tpr_result = evaluate_covariate_shift_detection(
            model=model,
            load_cifar10_corruption=load_cifar10_corruption,
            BasicDataset=BasicDataset,
            run_martingale=run_martingale,
            protector=protector_tpr,
            transform=transform,
            args=args,
            device=device,
            corruption_types=corruption_types,
            severities=severities,
            seeds=seeds_tpr,
            buffer_capacity=buffer_size,
            confidence_threshold=confidence_threshold,
            num_classes=10,
            use_pbrs=True,
            log_path=f'tpr_results_{run_id}.csv'
        )

        tpr_scores = [v['detection_rate'] for v in tpr_result.values()]
        delays = [v['avg_delay'] for v in tpr_result.values() if v['avg_delay'] is not None]
        mean_tpr = float(pd.Series(tpr_scores).mean())
        mean_delay = float(pd.Series(delays).mean()) if delays else 4000

        # --- FPR Evaluation ---
        fpr_result = compare_fpr_across_seeds(
            model=model,
            load_cifar10_label_shift_balanced=load_cifar10_label_shift_balanced,
            BasicDataset=BasicDataset,
            run_martingale=run_martingale,
            protector=protector_fpr,
            transform=transform,
            args=args,
            device=device,
            seeds=seeds_fpr,
            num_classes_list=num_classes_list,
            use_pbrs=True,
            log_path=f'fpr_results_{run_id}.csv'
        )

        fpr_scores = list(fpr_result.values())
        mean_fpr = float(pd.Series(fpr_scores).mean())

        # --- Scalar score (weighted trade-off) ---
        score = (0.8 * (1 - mean_fpr)) + (0.9 * mean_tpr) - (0.3 * (mean_delay / 4000))
        trial.set_user_attr("fpr", mean_fpr)
        trial.set_user_attr("tpr", mean_tpr)
        trial.set_user_attr("delay", mean_delay)

        results.append({
            'run_id': run_id,
            'trial': trial.number,
            'buffer_size': buffer_size,
            'confidence_threshold': confidence_threshold,
            'gamma': gamma,
            'eps_clip': eps_clip,
            'mean_fpr': mean_fpr,
            'mean_tpr': mean_tpr,
            'avg_delay': mean_delay,
            'score': score
        })

        print(f"🔎 Trial {trial.number}: score = {score:.3f} | FPR={mean_fpr:.3f} | TPR={mean_tpr:.3f} | Delay={mean_delay:.0f}")
        return score

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    df = pd.DataFrame(results)
    df.to_csv(log_path, index=False)
    print(f"\n✅ Optuna results saved to {log_path}")
    print("Best trial:", study.best_trial.params)
    return study


In [34]:
study = run_optuna_pbrs_optimization(
    model=model,
    compare_fpr_across_seeds=compare_fpr_across_seeds,
    evaluate_covariate_shift_detection=evaluate_covariate_shift_detection,
    load_cifar10_corruption=load_cifar10c,
    load_cifar10_label_shift_balanced=load_cifar10_label_shift,
    BasicDataset=BasicDataset,
    run_martingale=run_martingale,
    protector_factory=protector_factory,
    transform=transform,
    args=args,
    device=device,
    corruption_types=CORRUPTIONS,
    severities=[1, 2, 3, 4, 5],
    n_trials=40
)

[I 2025-05-11 01:21:52,418] A new study created in memory with name: no-name-62589858-b6e8-4337-9bff-38922eeadbd6
2025-05-11 01:21:52.420 | INFO     | protector:get_protector_from_ents:153 - creating protector from ents
2025-05-11 01:21:52.421 | INFO     | protector:set_gamma:49 - setting gamma val to 0.07216878364870323
2025-05-11 01:21:52.422 | INFO     | protector:set_eps_clip_val:45 - setting epsilon clip val to 1.8
2025-05-11 01:21:52.422 | INFO     | protector:get_protector_from_ents:153 - creating protector from ents
2025-05-11 01:21:52.423 | INFO     | protector:set_gamma:49 - setting gamma val to 0.07216878364870323
2025-05-11 01:21:52.424 | INFO     | protector:set_eps_clip_val:45 - setting epsilon clip val to 1.8



🔧 Trial 0: 20250511_012152_B256_T0.70_G0.072_E1.80
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 3 / 3 runs (rate = 1.00)  |  Avg delay: 193.0
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 3 / 3 runs (rate = 1.00)  |  Avg delay: 153.0
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 3 / 3 runs (rate = 1.00)  |  Avg delay: 148.0
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 3 / 3 runs (rate = 1.00)  |  Avg delay: 219.0
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 3 / 3 runs (rate = 1.00)  |  Avg delay: 254.0
Files already downloaded and verified
Files already downloaded and verified
Fi

[I 2025-05-11 01:47:05,792] Trial 0 finished with value: 1.2022385802469135 and parameters: {'buffer_size': 256, 'confidence_threshold': 0.7, 'gamma': 0.07216878364870323, 'eps_clip': 1.8}. Best is trial 0 with value: 1.2022385802469135.
2025-05-11 01:47:05.793 | INFO     | protector:get_protector_from_ents:153 - creating protector from ents
2025-05-11 01:47:05.794 | INFO     | protector:set_gamma:49 - setting gamma val to 0.14433756729740646
2025-05-11 01:47:05.795 | INFO     | protector:set_eps_clip_val:45 - setting epsilon clip val to 3.0
2025-05-11 01:47:05.795 | INFO     | protector:get_protector_from_ents:153 - creating protector from ents
2025-05-11 01:47:05.796 | INFO     | protector:set_gamma:49 - setting gamma val to 0.14433756729740646
2025-05-11 01:47:05.796 | INFO     | protector:set_eps_clip_val:45 - setting epsilon clip val to 3.0


[DEBUG] Max log_sj = 6.99 for subset size 9 with classes (0, 2, 3, 4, 5, 6, 7, 8, 9)

📊 FPR summary across seeds for method: PBRS
1 classes: PBRS → FPR = 0.667 ± 0.047
2 classes: PBRS → FPR = 0.533 ± 0.054
3 classes: PBRS → FPR = 0.300 ± 0.047
4 classes: PBRS → FPR = 0.067 ± 0.000
5 classes: PBRS → FPR = 0.033 ± 0.047
6 classes: PBRS → FPR = 0.011 ± 0.016
7 classes: PBRS → FPR = 0.022 ± 0.016
8 classes: PBRS → FPR = 0.022 ± 0.016
9 classes: PBRS → FPR = 0.967 ± 0.047

📊 Logging FPR results to fpr_results_20250511_012152_B256_T0.70_G0.072_E1.80.csv for method: PBRS
🔎 Trial 0: score = 1.202 | FPR=0.291 | TPR=0.720 | Delay=169

🔧 Trial 1: 20250511_012152_B32_T0.80_G0.144_E3.00
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 0 / 3 runs (rate = 0.00)  |  Avg delay: None
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 0 / 3 runs (rate

[I 2025-05-11 02:11:46,489] Trial 1 finished with value: 0.40759925925925933 and parameters: {'buffer_size': 32, 'confidence_threshold': 0.8, 'gamma': 0.14433756729740646, 'eps_clip': 3.0}. Best is trial 0 with value: 1.2022385802469135.
2025-05-11 02:11:46.490 | INFO     | protector:get_protector_from_ents:153 - creating protector from ents
2025-05-11 02:11:46.491 | INFO     | protector:set_gamma:49 - setting gamma val to 0.036084391824351615
2025-05-11 02:11:46.492 | INFO     | protector:set_eps_clip_val:45 - setting epsilon clip val to 1.5
2025-05-11 02:11:46.492 | INFO     | protector:get_protector_from_ents:153 - creating protector from ents
2025-05-11 02:11:46.493 | INFO     | protector:set_gamma:49 - setting gamma val to 0.036084391824351615
2025-05-11 02:11:46.493 | INFO     | protector:set_eps_clip_val:45 - setting epsilon clip val to 1.5


...

[DEBUG] Max log_sj = 4.30 for subset size 9 with classes (0, 2, 3, 4, 5, 6, 7, 8, 9)

📊 FPR summary across seeds for method: PBRS
1 classes: PBRS → FPR = 0.500 ± 0.082
2 classes: PBRS → FPR = 0.356 ± 0.068
3 classes: PBRS → FPR = 0.144 ± 0.031
4 classes: PBRS → FPR = 0.011 ± 0.016
5 classes: PBRS → FPR = 0.000 ± 0.000
6 classes: PBRS → FPR = 0.011 ± 0.016
7 classes: PBRS → FPR = 0.000 ± 0.000
8 classes: PBRS → FPR = 0.000 ± 0.000
9 classes: PBRS → FPR = 0.133 ± 0.047

📊 Logging FPR results to fpr_results_20250511_012152_B64_T0.50_G0.036_E2.00.csv for method: PBRS
🔎 Trial 37: score = 0.744 | FPR=0.128 | TPR=0.133 | Delay=981

🔧 Trial 38: 20250511_012152_B1024_T0.55_G0.072_E1.50
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 3 / 3 runs (rate = 1.00)  |  Avg delay: 168.0
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 3 / 3 runs (

[I 2025-05-11 22:38:11,565] Trial 38 finished with value: 1.3470397166105499 and parameters: {'buffer_size': 1024, 'confidence_threshold': 0.55, 'gamma': 0.07216878364870323, 'eps_clip': 1.5}. Best is trial 15 with value: 1.4300895957785742.
2025-05-11 22:38:11.570 | INFO     | protector:get_protector_from_ents:153 - creating protector from ents
2025-05-11 22:38:11.571 | INFO     | protector:set_gamma:49 - setting gamma val to 0.14433756729740646
2025-05-11 22:38:11.571 | INFO     | protector:set_eps_clip_val:45 - setting epsilon clip val to 3.0
2025-05-11 22:38:11.572 | INFO     | protector:get_protector_from_ents:153 - creating protector from ents
2025-05-11 22:38:11.573 | INFO     | protector:set_gamma:49 - setting gamma val to 0.14433756729740646
2025-05-11 22:38:11.573 | INFO     | protector:set_eps_clip_val:45 - setting epsilon clip val to 3.0


[DEBUG] Max log_sj = 6.99 for subset size 9 with classes (0, 2, 3, 4, 5, 6, 7, 8, 9)

📊 FPR summary across seeds for method: PBRS
1 classes: PBRS → FPR = 0.667 ± 0.047
2 classes: PBRS → FPR = 0.533 ± 0.054
3 classes: PBRS → FPR = 0.300 ± 0.047
4 classes: PBRS → FPR = 0.067 ± 0.000
5 classes: PBRS → FPR = 0.033 ± 0.047
6 classes: PBRS → FPR = 0.011 ± 0.016
7 classes: PBRS → FPR = 0.022 ± 0.016
8 classes: PBRS → FPR = 0.022 ± 0.016
9 classes: PBRS → FPR = 0.967 ± 0.047

📊 Logging FPR results to fpr_results_20250511_012152_B1024_T0.55_G0.072_E1.50.csv for method: PBRS
🔎 Trial 38: score = 1.347 | FPR=0.291 | TPR=0.880 | Delay=158

🔧 Trial 39: 20250511_012152_B32_T0.80_G0.144_E3.00
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 0 / 3 runs (rate = 0.00)  |  Avg delay: None
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 0 / 3 runs (r

[I 2025-05-11 23:03:16,492] Trial 39 finished with value: 0.40759925925925933 and parameters: {'buffer_size': 32, 'confidence_threshold': 0.8, 'gamma': 0.14433756729740646, 'eps_clip': 3.0}. Best is trial 15 with value: 1.4300895957785742.


[DEBUG] Max log_sj = 9.74 for subset size 9 with classes (0, 2, 3, 4, 5, 6, 7, 8, 9)

📊 FPR summary across seeds for method: PBRS
1 classes: PBRS → FPR = 0.700 ± 0.082
2 classes: PBRS → FPR = 0.700 ± 0.027
3 classes: PBRS → FPR = 0.478 ± 0.079
4 classes: PBRS → FPR = 0.256 ± 0.031
5 classes: PBRS → FPR = 0.089 ± 0.057
6 classes: PBRS → FPR = 0.233 ± 0.082
7 classes: PBRS → FPR = 0.211 ± 0.042
8 classes: PBRS → FPR = 0.167 ± 0.072
9 classes: PBRS → FPR = 1.000 ± 0.000

📊 Logging FPR results to fpr_results_20250511_012152_B32_T0.80_G0.144_E3.00.csv for method: PBRS
🔎 Trial 39: score = 0.408 | FPR=0.426 | TPR=0.067 | Delay=1489

✅ Optuna results saved to optuna_pbrs_results.csv
Best trial: {'buffer_size': 512, 'confidence_threshold': 0.5, 'gamma': 0.036084391824351615, 'eps_clip': 1.5}


In [33]:
study = run_optuna_pbrs_optimization(
    model=model,
    compare_fpr_across_seeds=compare_fpr_across_seeds,
    evaluate_covariate_shift_detection=evaluate_covariate_shift_detection,
    load_cifar10_corruption=load_cifar10c,
    load_cifar10_label_shift_balanced=load_cifar10_label_shift,
    BasicDataset=BasicDataset,
    run_martingale=run_martingale,
    protector_factory=protector_factory,
    transform=transform,
    args=args,
    device=device,
    corruption_types=CORRUPTIONS,
    severities=[1, 2, 3, 4, 5],
    n_trials=40
)

[I 2025-05-11 00:43:15,759] A new study created in memory with name: no-name-cabbe859-4fc2-47b9-9d77-a64662b31889
2025-05-11 00:43:15.761 | INFO     | protector:get_protector_from_ents:153 - creating protector from ents
2025-05-11 00:43:15.763 | INFO     | protector:set_gamma:49 - setting gamma val to 0.036084391824351615
2025-05-11 00:43:15.763 | INFO     | protector:set_eps_clip_val:45 - setting epsilon clip val to 2.5
2025-05-11 00:43:15.764 | INFO     | protector:get_protector_from_ents:153 - creating protector from ents


2025-05-11 00:43:15.765 | INFO     | protector:set_gamma:49 - setting gamma val to 0.036084391824351615
2025-05-11 00:43:15.766 | INFO     | protector:set_eps_clip_val:45 - setting epsilon clip val to 2.5



🔧 Trial 0: 20250511_004315_B256_T0.55_G0.036_E2.50
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 3 / 3 runs (rate = 1.00)  |  Avg delay: 210.0
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 3 / 3 runs (rate = 1.00)  |  Avg delay: 166.0
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 3 / 3 runs (rate = 1.00)  |  Avg delay: 165.0
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 3 / 3 runs (rate = 1.00)  |  Avg delay: 216.0
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 3 / 3 runs (rate = 1.00)  |  Avg delay: 222.0
Files already downloaded and verified
Files already downloaded and verified
Fi

[W 2025-05-11 01:08:38,804] Trial 0 failed with parameters: {'buffer_size': 256, 'confidence_threshold': 0.55, 'gamma': 0.036084391824351615, 'eps_clip': 2.5} because of the following error: TypeError("complex() first argument must be a string or a number, not 'list'").
Traceback (most recent call last):
  File "/opt/anaconda3/envs/poem/lib/python3.10/site-packages/pandas/core/nanops.py", line 1603, in _ensure_numeric
    x = float(x)
TypeError: float() argument must be a string or a real number, not 'list'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/envs/poem/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/_5/brt6j91s2fj6xh_3nslyydv40000gn/T/ipykernel_12737/2477026422.py", line 82, in objective
    mean_fpr = float(pd.Series(fpr_scores).mean())
  File "/opt/anaconda3/envs/poem/lib/python3.10/site-packages/pandas/cor

[DEBUG] Max log_sj = 4.30 for subset size 9 with classes (0, 2, 3, 4, 5, 6, 7, 8, 9)

📊 FPR summary across seeds for method: PBRS
1 classes: PBRS → FPR = 0.500 ± 0.082
2 classes: PBRS → FPR = 0.356 ± 0.068
3 classes: PBRS → FPR = 0.144 ± 0.031
4 classes: PBRS → FPR = 0.011 ± 0.016
5 classes: PBRS → FPR = 0.000 ± 0.000
6 classes: PBRS → FPR = 0.011 ± 0.016
7 classes: PBRS → FPR = 0.000 ± 0.000
8 classes: PBRS → FPR = 0.000 ± 0.000
9 classes: PBRS → FPR = 0.133 ± 0.047

📊 Logging FPR results to fpr_results_20250511_004315_B256_T0.55_G0.036_E2.50.csv for method: PBRS


TypeError: complex() first argument must be a string or a number, not 'list'

In [7]:
device

device(type='mps')

In [11]:
device = torch.device("mps")

In [9]:
source_ents

array([0.02236187, 0.0547286 , 0.02396516, ..., 0.05290807, 0.17147997,
       0.07488345])

In [16]:
combined_grid_search_pbrs(
    model=model,
    compare_fpr_across_seeds=compare_fpr_across_seeds,
    evaluate_covariate_shift_detection=evaluate_covariate_shift_detection,
    load_cifar10_corruption=load_cifar10c,
    load_cifar10_label_shift_balanced=load_cifar10_label_shift,
    BasicDataset=BasicDataset,
    run_martingale=run_martingale,
    protector_factory=lambda gamma, eps_clip, device: protect.get_protector_from_ents(
        source_ents,  # dummy, replaced later internally if needed
        args=argparse.Namespace(gamma=gamma, eps_clip=eps_clip, device=device)
    ),
    transform=transform,
    args=args,
    device=device,
    corruption_types=CORRUPTIONS,
    severities=[1, 2, 3, 4, 5],
    seeds_tpr=range(3),
    seeds_fpr=range(3),
    buffer_sizes=[32, 64, 128, 256, 512],
    confidence_thresholds=[0.5, 0.6, 0.7, 0.8, 0.9],
    gamma_values=[1 / (16 * 3 ** 0.5), 1 / (8 * 3 ** 0.5), 1 / (4 * 3 ** 0.5)],
    eps_clip_values=[1.5, 1.8, 2.0, 2.5, 3.0],
    log_path='combined_grid_search_results.csv'
)

2025-05-10 20:53:46.023 | INFO     | protector:get_protector_from_ents:153 - creating protector from ents
2025-05-10 20:53:46.027 | INFO     | protector:set_gamma:49 - setting gamma val to 0.036084391824351615
2025-05-10 20:53:46.028 | INFO     | protector:set_eps_clip_val:45 - setting epsilon clip val to 1.5


\n🔧 Running buffer=32, threshold=0.5, gamma=0.0361, eps_clip=1.50
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 0 / 3 runs (rate = 0.00)  |  Avg delay: None
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 0 / 3 runs (rate = 0.00)  |  Avg delay: None
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 0 / 3 runs (rate = 0.00)  |  Avg delay: None
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 0 / 3 runs (rate = 0.00)  |  Avg delay: None
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
✅ Detected in 0 / 3 runs (rate = 0.00)  |  Avg delay: None
Files already downloaded and verified
Files already downloaded and ve

2025-05-10 21:04:10.090 | INFO     | protector:get_protector_from_ents:153 - creating protector from ents
2025-05-10 21:04:10.091 | INFO     | protector:set_gamma:49 - setting gamma val to 0.036084391824351615
2025-05-10 21:04:10.092 | INFO     | protector:set_eps_clip_val:45 - setting epsilon clip val to 1.5


✅ Detected in 0 / 3 runs (rate = 0.00)  |  Avg delay: None

📊 Logging TPR results to tpr_results_20250510_205346_B32_T0.5_G0.036_E1.50.csv for method: PBRS

🔁 Running with seed: 0
[DEBUG] Max log_sj = 2.96 for subset size 1 with classes (5,)
[DEBUG] Max log_sj = 16.57 for subset size 2 with classes (0, 8)
[DEBUG] Max log_sj = 0.89 for subset size 3 with classes (5, 6, 9)
[DEBUG] Max log_sj = 2.34 for subset size 4 with classes (0, 7, 8, 9)
[DEBUG] Max log_sj = 0.32 for subset size 5 with classes (2, 5, 6, 7, 9)
[DEBUG] Max log_sj = 0.60 for subset size 6 with classes (1, 2, 5, 6, 7, 9)
[DEBUG] Max log_sj = 0.99 for subset size 7 with classes (0, 1, 2, 3, 5, 7, 8)
[DEBUG] Max log_sj = 0.84 for subset size 8 with classes (0, 1, 2, 4, 5, 6, 7, 9)
[DEBUG] Max log_sj = 4.60 for subset size 9 with classes (0, 1, 2, 3, 5, 6, 7, 8, 9)

🔁 Running with seed: 1
[DEBUG] Max log_sj = 6.20 for subset size 1 with classes (5,)
[DEBUG] Max log_sj = 11.00 for subset size 2 with classes (4, 9)
[DEBUG] Ma

AttributeError: 'NoneType' object has no attribute 'values'

📊 FPR summary across seeds for method: PBRS
1 classes: PBRS → FPR = 0.500 ± 0.082
2 classes: PBRS → FPR = 0.356 ± 0.068
3 classes: PBRS → FPR = 0.144 ± 0.031
4 classes: PBRS → FPR = 0.011 ± 0.016
5 classes: PBRS → FPR = 0.000 ± 0.000
6 classes: PBRS → FPR = 0.011 ± 0.016
7 classes: PBRS → FPR = 0.000 ± 0.000
8 classes: PBRS → FPR = 0.000 ± 0.000
9 classes: PBRS → FPR = 0.133 ± 0.047


📊 Logging FPR results to fpr_results_20250510_205346_B32_T0.5_G0.036_E1.50.csv for method: PBRS


In [18]:
import itertools
import pandas as pd
from datetime import datetime
from joblib import Parallel, delayed

def run_single_grid_combo(model, buffer_size, threshold, gamma, eps_clip,
                          compare_fpr_across_seeds, evaluate_covariate_shift_detection,
                          load_cifar10_corruption, load_cifar10_label_shift_balanced,
                          BasicDataset, run_martingale, protector_factory,
                          transform, args, device, corruption_types, severities,
                          seeds_tpr, seeds_fpr, num_classes_list):

    run_id = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_B{buffer_size}_T{threshold}_G{gamma:.3f}_E{eps_clip:.2f}"
    print(f"--> [Grid] buffer={buffer_size}, threshold={threshold}, gamma={gamma}, eps_clip={eps_clip}")

    # TPR
    protector_tpr = protector_factory(gamma=gamma, eps_clip=eps_clip, device=device)
    tpr_result = evaluate_covariate_shift_detection(
        model=model,
        load_cifar10_corruption=load_cifar10_corruption,
        BasicDataset=BasicDataset,
        run_martingale=run_martingale,
        protector=protector_tpr,
        transform=transform,
        args=args,
        device=device,
        corruption_types=corruption_types,
        severities=severities,
        seeds=seeds_tpr,
        buffer_capacity=buffer_size,
        confidence_threshold=threshold,
        num_classes=10,
        use_pbrs=True,
        log_path=None
    )
    tpr_scores = [v['detection_rate'] for v in tpr_result.values()]
    delays = [v['avg_delay'] for v in tpr_result.values() if v['avg_delay'] is not None]
    mean_tpr = float(pd.Series(tpr_scores).mean())
    std_tpr = float(pd.Series(tpr_scores).std())
    mean_delay = float(pd.Series(delays).mean()) if delays else None
    std_delay = float(pd.Series(delays).std()) if delays else None

    # FPR
    protector_fpr = protector_factory(gamma=gamma, eps_clip=eps_clip, device=device)
    fpr_result = compare_fpr_across_seeds(
        model=model,
        load_cifar10_label_shift_balanced=load_cifar10_label_shift_balanced,
        BasicDataset=BasicDataset,
        run_martingale=run_martingale,
        protector=protector_fpr,
        transform=transform,
        args=args,
        device=device,
        seeds=seeds_fpr,
        num_classes_list=num_classes_list,
        use_pbrs=True,
        log_path=None
    )
    fpr_scores = list(fpr_result.values())
    mean_fpr = float(pd.Series(fpr_scores).mean())
    std_fpr = float(pd.Series(fpr_scores).std())

    score = (0.8 * (1 - mean_fpr)) + (0.9 * mean_tpr) - (0.3 * (mean_delay / 4000)) if mean_delay is not None else None

    return {
        'run_id': run_id,
        'buffer_size': buffer_size,
        'confidence_threshold': threshold,
        'gamma': gamma,
        'eps_clip': eps_clip,
        'mean_fpr': mean_fpr,
        'std_fpr': std_fpr,
        'mean_tpr': mean_tpr,
        'std_tpr': std_tpr,
        'avg_delay': mean_delay,
        'std_delay': std_delay,
        'score': score
    }

def combined_grid_search_pbrs_parallel(model, compare_fpr_across_seeds, evaluate_covariate_shift_detection,
                                       load_cifar10_corruption, load_cifar10_label_shift_balanced,
                                       BasicDataset, run_martingale, protector_factory,
                                       transform, args, device, corruption_types, severities,
                                       seeds_tpr=range(3), seeds_fpr=range(3),
                                       num_classes_list=[1,2,3,4,5,6,7,8,9, 10],
                                       buffer_sizes=[32, 64, 128, 256, 512],
                                       confidence_thresholds=[0.5, 0.6, 0.7, 0.8, 0.9],
                                       gamma_values=[1 / (16 * 3 ** 0.5), 1 / (8 * 3 ** 0.5), 1 / (4 * 3 ** 0.5)],
                                       eps_clip_values=[1.5, 1.8, 2.0, 2.5, 3.0],
                                       log_path='combined_grid_search_parallel_results.csv',
                                       n_jobs=8):

    grid = list(itertools.product(buffer_sizes, confidence_thresholds, gamma_values, eps_clip_values))
    results = Parallel(n_jobs=n_jobs)(delayed(run_single_grid_combo)(
        model, b, t, g, e,
        compare_fpr_across_seeds, evaluate_covariate_shift_detection,
        load_cifar10_corruption, load_cifar10_label_shift_balanced,
        BasicDataset, run_martingale, protector_factory,
        transform, args, device, corruption_types, severities,
        seeds_tpr, seeds_fpr, num_classes_list
    ) for b, t, g, e in grid)

    df = pd.DataFrame(results)
    df.to_csv(log_path, index=False)
    print(f"\n✅ Parallel grid search results saved to: {log_path}")

In [22]:
device = torch.device("cpu")
def protector_factory(gamma, eps_clip, device):
    return protect.get_protector_from_ents(
        source_ents=[],  # only used internally for CDF if needed
        args=argparse.Namespace(gamma=gamma, eps_clip=eps_clip, device=device)
    )
def model_factory():
    model = get_model(args.method, device=torch.device("cpu"))  # Rebuild model on CPU
    #model.load_state_dict(torch.load("model_weights.pth", map_location="cpu"))  # Load weights
    model.eval()
    return model

combined_grid_search_pbrs_parallel(
    model=model_factory(),
    compare_fpr_across_seeds=compare_fpr_across_seeds,
    evaluate_covariate_shift_detection=evaluate_covariate_shift_detection,
    load_cifar10_corruption=load_cifar10c,
    load_cifar10_label_shift_balanced=load_cifar10_label_shift,
    BasicDataset=BasicDataset,
    run_martingale=run_martingale,
    protector_factory=protector_factory,
    transform=transform,
    args=args,
    device=device,
    corruption_types=CORRUPTIONS,
    severities=[1, 2, 3, 4, 5],
    seeds_tpr=range(3),
    seeds_fpr=range(3),
    num_classes_list=[1, 2, 3, 4, 5, 6, 7, 8, 9,10],
    buffer_sizes=[32, 64, 128, 256],  # keep small initially to test
    confidence_thresholds=[0.6, 0.7, 0.8],
    gamma_values=[1 / (16 * 3 ** 0.5), 1 / (8 * 3 ** 0.5), 1 / (4 * 3 ** 0.5)],
    eps_clip_values=[1.2, 1.5, 1.8, 2.0],
    log_path='combined_grid_search_parallel_results.csv',
    n_jobs=8  # M1 Max has 8 performance cores
)

Using cache found in /Users/louis/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


PicklingError: Could not pickle the task to send it to the workers.